# Experiment for constrained forecast reconciliation with Tourism Dataset


Base forecast is generated by `auto.arima` of `forecast` package in R.

Tourism Dataset containes 76 bottom series, which construct a hierarchy of 3 levels. Time series of level 2 are region, level1 are state and level0 is total.

I choose RMSE as forecast accuracy measure.

In [1]:
import pandas as pd
import numpy as np
import pyhts.reconciliation as fr

## Prepare base forecast and true value

In [12]:
base_forecast = pd.read_csv('tourism_base_96.csv')

In [14]:
new_bf = pd.DataFrame()
new_bf["index"] = base_forecast["X2"]
new_bf["id"] = base_forecast["X1"]
new_bf[list(map(lambda x: 'y' + str(x+1), range(96)))] = base_forecast.iloc[:, 3: (3+96)]
new_bf[list(map(lambda x: 'h' + str(x+1), range(12)))] = base_forecast.iloc[:, (96+3): (96+3+12)]

In [22]:
base_forecast = new_bf
del new_bf

In [55]:
# 准备真实值
toursim_raw = pd.read_csv("data/TourismData_v4.csv")
from pyhts.hts import Hts
toursim = Hts.from_hts(toursim_raw, characters=[1,1,1], m=12)
toursim_all = toursim.aggregate_ts()

In [29]:
insample_columns = list(map(lambda x: 'y' + str(x+1), range(96)))
forecast_columns = list(map(lambda x: 'h' + str(x+1), range(12)))

In [32]:
def cal_se(x, h=12):
    y_pred = x[forecast_columns].values
    series = int(x['id'])
    T = int(x['index'])
    y_true = toursim_all[T:(T+h), series-1]
    return np.square(y_pred-y_true)

In [53]:
def gen_ses_matrix(bf):
    sse = bf[bf['index']<=228].apply(cal_se, axis=1)
    ses = pd.DataFrame()
    ses['id'] = bf[bf['index']<=228]['id']
    ses['index'] = bf[bf['index']<=228]['index']
    ses['level'] = ses['id'].map(lambda x: toursim.node_level[int(x)-1])
    ses[forecast_columns] = np.stack(sse, axis=0)
    return ses

In [54]:
ses_base = gen_ses_matrix(base_forecast)

## gennerate reconciled forecast for all methods

In [71]:
def cal_reconcilded_forecat(bf, keep_fitted=False, **kwargs):
    index = bf["index"].iloc[0]
    if keep_fitted:
        base_forecast = bf[insample_columns+forecast_columns].values.T
    else:
        base_forecast = bf[forecast_columns].values.T
    hist_hts = Hts.from_hts(toursim_raw[(index-96): index], characters=[1,1,1], m=12)
    reconciled_all = fr.wls(hist_hts, base_forecast, **kwargs).aggregate_ts().T
    res = pd.DataFrame()
    res["index"] = bf["index"]
    res["id"] = bf["id"]
    res[forecast_columns] = reconciled_all
    return res

In [72]:
forecast_ols = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, method="ols"))

In [76]:
forecast_cls = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, method="ols", 
                                                                                      constraint=True))

In [78]:
forecast_mint = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, keep_fitted=True,
                                                                                       method="mint", weighting="shrink"))
forecast_var = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, keep_fitted=True,
                                                                                       method="mint", weighting="var"))

In [80]:
forecast_cmint = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, keep_fitted=True,
                                                                                       method="mint", weighting="shrink", constraint=True))
forecast_cvar = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, keep_fitted=True,
                                                                                       method="mint", weighting="var", constraint=True))

In [149]:
forecast_nseries = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, method="wls", weighting="nseries", 
                                                                                      constraint=False))
forecast_cnseries = base_forecast.groupby('index').apply(lambda x: cal_reconcilded_forecat(x, method="wls", weighting="nseries", 
                                                                                      constraint=True))

## Generate Report

In [148]:
def sse_report(sse, h=1):
    report = sse.groupby('level').apply(
        lambda x: np.sqrt(np.mean(x.iloc[:, 3:(3+h)]).mean())
    )
    return report

def gen_reports(report_matrix, forecast_mat, name):
    report[name] = 0
    idx=pd.IndexSlice
    ses = gen_ses_matrix(forecast_mat)
    for h in [1, 4, 6, 12]:
        h_re = sse_report(ses, h=h)
        report.loc[idx[h, :], name] = h_re.values
    return report

In [145]:
index = pd.MultiIndex.from_product([[1, 4, 6, 12], range(4)], names=["h", "level"])
report = pd.DataFrame(index=index)

In [150]:
gen_reports(report, base_forecast, "base")
gen_reports(report, forecast_ols, "ols")
gen_reports(report, forecast_cls, "cls")
gen_reports(report, forecast_mint, "mint_shrinkage")
gen_reports(report, forecast_cmint, "c mint_shrinkage")
gen_reports(report, forecast_var, "mint var")
gen_reports(report, forecast_cvar, "c mint var")
gen_reports(report, forecast_nseries, "nseries")
gen_reports(report, forecast_nseries, "c nseries")

base          ols          cls  mint_shrinkage  \
h  level                                                          
1  0      1809.700771  1783.350555  1809.700771     1774.904824   
   1       502.999421   497.674686   499.601809      494.715908   
   2       217.460154   214.157131   214.503748      212.043177   
   3       119.648795   118.937085   119.023953      118.326301   
4  0      1805.268455  1789.312078  1805.268455     1791.766793   
   1       507.139275   499.599684   500.745497      496.213204   
   2       218.886006   214.773657   214.990232      212.107869   
   3       119.357268   118.696771   118.751937      118.004872   
6  0      1814.605461  1802.373861  1814.605461     1813.939527   
   1       510.602116   502.603107   503.466356      499.731793   
   2       219.566486   215.403074   215.575730      212.854300   
   3       119.444211   118.698126   118.742826      118.079162   
12 0      1891.145282  1880.670397  1891.145282     1895.264612   
   1       522.996286   514.649273   515.360239      511.271128   
   2       220.546986   216.645800   216.813627      213.881938   
   3       120.017725   119.225721   119.272806      118.535918   

          c mint_shrinkage     mint var   c mint var      nseries    c nseries  
h  level                                                                        
1  0           1809.700771  1773.662525  1809.700771  1758.310302  1758.310302  
   1            501.078204   493.596190   501.846782   491.933938   491.933938  
   2            213.643740   212.771989   215.966425   212.093968   212.093968  
   3            118.655850   118.520741   119.281862   118.517791   118.517791  
4  0           1805.268455  1800.304800  1805.268455  1784.080872  1784.080872  
   1            500.291300   496.593468   501.271208   495.144574   495.144574  
   2            213.213848   213.406068   215.861416   212.811175   212.811175  
   3            118.236405   118.368866   118.967207   118.327044   118.327044  
6  0           1814.605461  1822.274000  1814.605461  1804.717087  1804.717087  
   1            502.219517   500.522409   503.561577   499.049016   499.049016  
   2            213.626240   214.229729   216.327227   213.651397   213.651397  
   3            118.237263   118.474278   118.987081   118.405621   118.405621  
12 0           1891.145282  1911.098665  1891.145282  1890.683379  1890.683379  
   1            512.597520   513.138731   514.157352   511.376830   511.376830  
   2            214.537187   215.538061   217.327699   214.951096   214.951096  
   3            118.648404   118.986334   119.397863   118.936151   118.936151